<a href="https://colab.research.google.com/github/shiloh-parker/llm-20-questions/blob/main/20-questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COSC-523 P2 Group 13: 20 Questions
### Jared Bell, Imran Dewjee, Yair Elovic, Shiloh Parker

In [3]:
!git clone https://github.com/shiloh-parker/llm-20-questions/
%cd llm-20-questions-main
!ls -la

Cloning into 'llm-20-questions'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 48 (delta 11), reused 22 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 3.40 MiB | 4.08 MiB/s, done.
Resolving deltas: 100% (11/11), done.
[Errno 2] No such file or directory: 'llm-20-questions-main'
/content
total 20
drwxr-xr-x 1 root root 4096 Nov 23 20:16 .
drwxr-xr-x 1 root root 4096 Nov 23 20:10 ..
drwxr-xr-x 4 root root 4096 Nov 20 14:30 .config
drwxr-xr-x 5 root root 4096 Nov 23 20:16 llm-20-questions
drwxr-xr-x 1 root root 4096 Nov 20 14:30 sample_data


## Loading Model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="auto", #Mark MPS for M series Mac's, cuda for NVIDIA gpus
    dtype="auto",
    trust_remote_code=False,
    temperature=0
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Create a pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_length=4000,
    max_new_tokens=15,
    do_sample=False,
)


/Users/jared/.pyenv/versions/3.10.16/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.78s/it]
Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


## Generating Decision Tree Questions from Properties

In [ ]:
with open('noun_universe.txt', 'r') as f:
    nouns = [line.rstrip() for line in f]

with open('property_universe.txt', 'r') as f:
    properties = [line.rstrip() for line in f]

property_questions = {}

for prop in properties:
    prompt = [
        {"role": "system", "content": "You are building a series of questions to ask in a game of 20 questions where are user will provide Yes/No answers. You will not actually be asked these questions, but based on rankings of nouns, a binary decision tree will be created using your provided questions to determine the best guess. You will base the generated question from a single provided property. Provide only a single yes or no question to ask the user for the provided property. Provide no additional context or words."
        },
        {"role": "user", "content": f"Create a question to help sort a list of things based on yes or no answers. The question should be based on this property: {prop}"},
    ]
    question = pipe(prompt)[0]["generated_text"]
    print(question)
    property_questions[prop] = question

print(len(property_questions))


 Is the item easily accessible to people with disabilities?
 Is the item accessible globally?
 Is the item known for its high level of precision?
 Is the item known for its precision and correctness?
 "Is the item associated with high physical activity?"
 Is the item known for its aggressive behavior or characteristics?
 Is the item known for its agility?
 "Does the item shed a significant amount of hair or fur?"
 "Is the annual precipitation above 1000mm?"
 Is the item considered visually appealing?
 Is the item adapted for living in water?
 Is the item fragrant?
 Is the item artificially created?
 Is it capable of attacking?
 "Is the content suitable for a general audience?"
 Is it considered beautiful?
 Is the item considered beautiful?
 Is it big?
 Is the item rich in biodiversity?
 Is the object larger than a basketball?
 Is the item breakable?
 Is the item bright?
 Is the item designed to blend in with its surroundings?
 "Can the item hold a significant amount of content?"
 "Is t

In [ ]:
import json

with open('property_questions.txt', 'w') as f:
    f.write(json.dumps(property_questions, indent=4))


## Property Generation

## Property Parsing

In [ ]:
import json

file = open("all_properties_question_scored.json", "r")
data = file.read()
json_data = json.loads(data)

formatted_data = {}

for prop, scores in json_data.items():
    formatted_data[prop] = {}
    for score in scores:
        noun = score[0]
        value = score[1]
        formatted_data[prop][noun] = value

property_table = {}
threshold = 5
for prop, noun_scores in formatted_data.items():
    property_table[prop] = []
    for noun, score in noun_scores.items():
        if score>=threshold:
            property_table[prop].append(noun)

json_str = json.dumps(property_table)

with open("property_table.json", "w") as f:
    f.write(json_str)


## Example Property Comparison

In [ ]:
import copy

property_json = open("property_table.json", "r").read()

property_table = json.loads(property_json)

# Using sets for easier comparison of noun lists
set_table = {k: set(v) for k, v in property_table.items()}

remainder = copy.deepcopy(set_table)
# No response to question
subtraction = set_table["accessibility"]
rem_table = {k: v-subtraction for k, v in remainder.items() if len(v-subtraction)>0}
remainder = rem_table

print(len(rem_table))

# Yes response to question
intersection = set_table["accuracy"]
rem_table = {k: v & intersection for k, v in remainder.items() if len(v & intersection)>0}
remainder = rem_table

print(len(rem_table))


227
227


###Data Load

In [12]:
import json
import numpy as np

with open('property_table.json', 'r') as f:
    property_table = json.load(f)

with open('noun_universe.txt', 'r') as f:
    all_nouns = [line.strip() for line in f]

with open('questions.json', 'r') as f:
    data = json.load(f)
    property_questions = data['questions']

print(f"Loaded {len(property_questions)} questions")
print(f"Loaded {len(property_table)} properties")
print(f"Loaded {len(all_nouns)} nouns")

Loaded 228 questions
Loaded 228 properties
Loaded 1852 nouns


###Information Gain Helper Functions

In [13]:
# Core entropy Calculation, how many questions you'd need to identify one item
def calculate_entropy(n):
    if n <= 1:
        return 0
    return np.log2(n)

# Information gain calculation: how much uncertainty is reduced by asking this question
def calculate_information_gain_fast(num_candidates, num_yes, num_no):
    if num_candidates <= 1:
        return 0

    current_entropy = calculate_entropy(num_candidates)

    # Weighted average of entropies after split
    yes_entropy = calculate_entropy(num_yes) * (num_yes / num_candidates)
    no_entropy = calculate_entropy(num_no) * (num_no / num_candidates)

    info_gain = current_entropy - (yes_entropy + no_entropy)
    return info_gain

# Question selection: finds the property that maximizes information gain (best split)
def select_best_question(candidates, property_table, property_questions):
    candidates_set = set(candidates)  # O(1) lookups
    best_property = None
    best_gain = -1

    for property_name in property_questions.keys():
        # Fast set intersection
        nouns_with_property = set(property_table.get(property_name, []))
        num_yes = len(candidates_set & nouns_with_property)  # Set intersection
        num_no = len(candidates_set) - num_yes

        gain = calculate_information_gain_fast(len(candidates_set), num_yes, num_no)

        if gain > best_gain:
            best_gain = gain
            best_property = property_name

    return best_property, best_gain

###Game Loop

In [ ]:
# Core game runner
def play_game(answer_noun, property_table, property_questions, all_nouns, max_questions=20, verbose=True):
    candidates = all_nouns.copy()
    nouns_with_property_cache = {prop: set(property_table.get(prop, [])) for prop in property_questions.keys()}

    for question_num in range(1, max_questions + 1):
        if len(candidates) == 1:
            if verbose:
                print(f"\nFound it in {question_num - 1} questions: {candidates[0]}")
            return question_num - 1, candidates[0] == answer_noun

        # Select best question
        best_property, gain = select_best_question(candidates, property_table, property_questions)

        if verbose:
            print(f"\nQ{question_num}: {property_questions[best_property]}")
            print(f"   Property: {best_property}, Info gain: {gain:.3f}, Candidates: {len(candidates)}")

        # Determine the correct answer for this noun
        answer_is_yes = answer_noun in nouns_with_property_cache[best_property]

        if verbose:
            print(f"   Answer: {'YES' if answer_is_yes else 'NO'}")

        # Filter candidates based on answer
        candidates_set = set(candidates)
        if answer_is_yes:
            candidates = list(candidates_set & nouns_with_property_cache[best_property])
        else:
            candidates = list(candidates_set - nouns_with_property_cache[best_property])

    # Out of questions make final guess (most common or first remaining)
    final_guess = candidates[0] if candidates else all_nouns[0]
    if verbose:
        print(f"\nLast question! Are you thinking of {final_guess}?")
        print(f"Correct answer was: {answer_noun}")

    return max_questions, final_guess == answer_noun

# Initial Game Test
test_noun = "computer"
num_q, success = play_game(test_noun, property_table, property_questions, all_nouns)
print(f"\n{'EXACTLY CORRECT' if success else 'WRONG'} - took {num_q} questions")


Q1: Output ONLY a single integer 1-10 (no words). Rate how true the following statement is: Is the item electrical?
   Property: electrical, Info gain: 1.000, Candidates: 1852
   Answer: NO

Q2: Output ONLY a single integer 1-10 (no words). Rate how true the following statement is: Is the item heavier than a standard textbook?
   Property: weight, Info gain: 1.000, Candidates: 923
   Answer: YES

Q3: Output ONLY a single integer 1-10 (no words). Rate how true the following statement is: Is the item portable and can be easily carried by one person?
   Property: portability_1_10, Info gain: 1.000, Candidates: 465
   Answer: YES

Q4: Output ONLY a single integer 1-10 (no words). Rate how true the following statement is: Is the item strong?
   Property: strong, Info gain: 0.998, Candidates: 234
   Answer: YES

Q5: Output ONLY a single integer 1-10 (no words). Rate how true the following statement is: Is it difficult to perform?
   Property: difficult, Info gain: 1.000, Candidates: 123
   

###10 Test Games

In [ ]:
import random

# Run the 10 (or however many desired test games)
def run_test_games(num_games=10):
    results = []
    test_nouns = random.sample(all_nouns, num_games)

    print(f"Running {num_games} test games...\n")
    print("="*60)

    for i, noun in enumerate(test_nouns, 1):
        print(f"\n### GAME {i}: Target = '{noun}' ###")
        num_q, success = play_game(noun, property_table, property_questions, all_nouns, verbose=False)
        results.append((noun, num_q, success))
        print(f"Result: {num_q} questions - {'✓ EXACTLY CORRECT' if success else '✗ WRONG'}")

    print("\n" + "="*60)
    print("SUMMARY:")
    print("="*60)

    successes = sum(1 for _, _, s in results if s)
    total_questions = sum(q for _, q, _ in results)
    avg_questions = total_questions / num_games

    print(f"Success rate: {successes}/{num_games} ({100*successes/num_games:.1f}%)")
    print(f"Average questions: {avg_questions:.2f}")
    print(f"Min questions: {min(q for _, q, _ in results)}")
    print(f"Max questions: {max(q for _, q, _ in results)}")

    return results

results = run_test_games(10)

Running 10 test games...


### GAME 1: Target = 'wall sit' ###
Result: 11 questions - ✓ SUCCESS

### GAME 2: Target = 'weighted vest squats' ###
Result: 11 questions - ✓ SUCCESS

### GAME 3: Target = 'gorilla' ###
Result: 11 questions - ✓ SUCCESS

### GAME 4: Target = 'coffee' ###
Result: 11 questions - ✓ SUCCESS

### GAME 5: Target = 'pyramids of giza' ###
Result: 11 questions - ✓ SUCCESS

### GAME 6: Target = 'oak' ###
Result: 11 questions - ✓ SUCCESS

### GAME 7: Target = 'sushi roll' ###
Result: 11 questions - ✓ SUCCESS

### GAME 8: Target = 'amazon' ###
Result: 11 questions - ✓ SUCCESS

### GAME 9: Target = 'marie antoinette' ###
Result: 11 questions - ✓ SUCCESS

### GAME 10: Target = 'dispatcher' ###
Result: 11 questions - ✓ SUCCESS

SUMMARY:
Success rate: 10/10 (100.0%)
Average questions: 11.00
Min questions: 11
Max questions: 11


###Interactive Game Player

In [ ]:
def quickplay(noun):
    if noun not in all_nouns:
        print(f"ERROR: '{noun}' is not in the noun universe!")
        return None

    print(f"\n🎮 Playing: '{noun}'")
    print("="*60)
    num_q, success = play_game(noun, property_table, property_questions, all_nouns, verbose=True)
    print("="*60)
    print(f"✓ EXACTLY CORRECT in {num_q} questions!" if success else f"✗ WRONG after {num_q} questions")
    return num_q

### Enter the noun below and run the cell
quickplay("pizza")


🎮 Playing: 'pizza'

Q1: Output ONLY a single integer 1-10 (no words). Rate how true the following statement is: Is the item electrical?
   Property: electrical, Info gain: 1.000, Candidates: 1852
   Answer: NO

Q2: Output ONLY a single integer 1-10 (no words). Rate how true the following statement is: Is the item heavier than a standard textbook?
   Property: weight, Info gain: 1.000, Candidates: 923
   Answer: NO

Q3: Output ONLY a single integer 1-10 (no words). Rate how true the following statement is: Is the object's mass greater than 1 kilogram?
   Property: mass, Info gain: 1.000, Candidates: 458
   Answer: NO

Q4: Output ONLY a single integer 1-10 (no words). Rate how true the following statement is: Is it fast?
   Property: fast, Info gain: 1.000, Candidates: 226
   Answer: NO

Q5: Output ONLY a single integer 1-10 (no words). Rate how true the following statement is: Is the cost of living high in this location?
   Property: cost of living, Info gain: 1.000, Candidates: 113
  

11

### Overall Approach and Design Process
> First, similarly to the individual homework assignment on this very topic, our code loaded the *Phi 3 mini* model from Microsoft. As instructed, we used this model to solely generate questions based on provided properties before initiating our game. We used that model to begin the long process of scoring each noun by each property, storing the results in a JSON file. For the game setup, we calculated entropy to weigh the efficacy of questions, with this being based on log base 2. Using entropy, we could determine how much uncertainty was reduced by a potential question. This uses our important technique of a tree structure. We would select the best question available each time; in theory, if we had good questions at our disposal, the max amount of questions would be log base 2 of our nouns. The game itself iterated through these reclassifications until a result or 20 questions was met.
> Initially, we planned to use random questions, but we found performance to be lackluster. This would have included some sort of backtracking if fully fleshed out, but Jared brought up the incredible idea of using a tree structure.

### Overall Performance
> Wow, I think our team is very content with the performance of this project. This has been a tremendously daunting task, especially considering the fact that many groups have struggled to make a dent. In our studies, our model guessed "Exactly Correct" every time, as shown by our test games. Every single example of ours was able to guess the model in 11 questions. Now, as we know, 2^11 is 2048. This is the lowest power of two that hits our nouns threshold, which is right under 2000. This means our questions were fairly consistent in asking decisions that split the dataset, as evidenced in the "Overall Approach and Design Process."

### Future Work and Limitations
> For our model, we did not find many limitations, as we intentionally created it in a way that was quite "low risk" as it turns the problem into binary search. With that said, our way of coding does potentially limit extremely quick guesses, although I think we would all agree that this tradeoff is necessary. In a job, we would prefer the consistency of continually passing the assignment of using under 20 questions. In the future, it is possible that performance could be improved, of course, by improving the quality of questions. As many have likely noted, this particular LLM is quite limited in its understanding, so a newer LLM for question generation may result in better results. If given more time, since use of an LLM is recommended but not exclusive, I believe it would be beneficial to manually go through each generated noun-property rating. Having good, consistent answers can only ever improve performance, and I know that answers did not always seem consistent (such as rock with a weight of 4 but a bear cub with a weight of 2) during the related individual homework assignment. As well, I would further refine the accuracy of our correctness tiers.